In [1]:
PRFX='prep0318_2'

# setup

In [2]:
from pathlib import Path
import pickle
import dask.dataframe as dd
import pandas as pd
import dask
print('dask.__version__', dask.__version__)
import xgboost
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwing_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_follower_cnt',
'u2_following_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]

cols_val = cols[:-4]

cols_tgt_tmstmp=[
    'reply_tm',
    'retwt_tm',
    'retwt_cmmnt_tm',
    'like_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
print(cols_tgt)

dask.__version__ 2.12.0
['reply', 'retwt', 'retwt_cmmnt', 'like']


# prepare data

In [3]:
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/trn.tsv
#  148,075,238 data/trn.tsv
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/val.tsv
#  15,127,684 data/val.tsv

In [10]:
ls $p_in | grep val

val_0331BF70E606D62D92C96CE9AD71A7CF_1584405047.tsv
val_06BEAB41D66CCFF329D1ED8BA120A6C2_1584405047.tsv
val_06D61DCBBE938971E1EA0C38BD9B5446_1584405047.tsv
val_12D8CEB94F89D11D7EB95EAE9689B009_1584405047.tsv
val_1584405047_1e3.tsv
val_1584405047.tsv*
val_1584626395.tsv
val_1BC639981AE88E09129594B11F894A21_1584405047.tsv
val1e4.tsv*
val_1FFD2FE4297F5E70EBC6C3230D95CB9C_1584405047.tsv
val_2216D01F7B48554E4211021A46816FCF_1584405047.tsv
val_22C448FF81263D4BAF2A176145EE9EAD_1584405047.tsv
val_259A6F6DFD672CB1F883CBEC01B99F2D_1584405047.tsv
val_2E18F6F53E3CF073911AF0A93BBE5373_1584405047.tsv
val_3121F7240D488F74EEED9312E174B217_1584405047.tsv
val_3A85BCEC571C3F5AB1069E4924189177_1584405047.tsv
val_4249CE88433AEA3F8DCEECF008B3CB95_1584405047.tsv
val_4690215948DBF6872B8ED1C2BC87B17E_1584405047.tsv
val_48236EC80FDDDFADE99420ABC9210DDF_1584405047.tsv
val_515E873C86EE1577E75FA2387B7FA59E_1584405047.tsv
val_54208B51D44E7D91DC2F3DD02ADEDEC2_1584405047.tsv
val_544FA32458C903F1125FE6598300A047_15844

In [11]:
trntmstmp=1584412344
datetime.datetime.fromtimestamp(trntmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-17 02:32:24'

In [12]:
valtmstmp=1584626395
datetime.datetime.fromtimestamp(valtmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-19 13:59:55'

In [13]:
ls $p_in -hlS | grep {valtmstmp}

-rw-rw-r-- 1 ubuntu ubuntu  6.6G Mar 16 13:56 val_1584626395.tsv


In [7]:
lang='3A85BCEC571C3F5AB1069E4924189177'

In [8]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

dfval=dd.read_csv(
    f'{p_in}/val_{lang}_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)

In [9]:
dftrn[cols_tgt]=~dftrn[cols_tgt_tmstmp].isna().astype('int8')+2
display(dftrn[cols_tgt_tmstmp].head())
ys=dftrn[cols_tgt]
ys.head()

,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
0,NaN,NaN,NaN,1.581246e+09
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.581509e+09
3,NaN,NaN,NaN,1.581082e+09
4,1.581234e+09,NaN,NaN,1.581234e+09


,reply,retwt,retwt_cmmnt,like
0,0,0,0,1
1,0,0,0,0
2,0,0,0,1
3,0,0,0,1
4,1,0,0,1


In [10]:
df = dfval
df['len_toks']=df.toks.apply(len, meta=('toks', 'int64'))
for media in ['Photo', 'Video', 'GIF']:
    df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x, 
                                                                 meta=('media', 'string'))
for col in ['hshtgs', 'links', 'domns',]:
    df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0,
                                                      meta=(col, 'string'))
df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
df['tmdlta_twtu2'] = df.tm - df.u2_create_tm


In [11]:
# dfval.twttyp.value_counts().compute()
# TopLevel    2932
# Retweet      994
# Quote        213
twttyps=['TopLevel','Retweet','Quote']
for typ in twttyps:
    df[f'twttyp_{typ}']=(df.twttyp==typ).astype('int8')

df[['twttyp']+[f'twttyp_{typ}' for typ in twttyps]].head()

,twttyp,twttyp_TopLevel,twttyp_Retweet,twttyp_Quote
0,TopLevel,1,0,0
1,Retweet,0,1,0
2,TopLevel,1,0,0
3,TopLevel,1,0,0
4,TopLevel,1,0,0


In [14]:
def prep(df, isval=True):
    df['len_toks']=df.toks.apply(len, meta=('toks', 'int64'))
    for media in ['Photo', 'Video', 'GIF']:
        df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x, 
                                                                     meta=('media', 'string'))
    for col in ['hshtgs', 'links', 'domns',]:
        df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0,
                                                          meta=(col, 'string'))
    for typ in ['TopLevel','Retweet','Quote']:
        df[f'twttyp_{typ}']=(df.twttyp==typ).astype('int8')

    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
    df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
    df['tmdlta_twtu2'] = df.tm - df.u2_create_tm
    
    if isval: return df
    
    df[cols_tgt]=~df[cols_tgt_tmstmp].isna().astype('int8')+2
    return df

In [13]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

dfval=dd.read_csv(
    f'{p_in}/val_{lang}_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)

dftrn = prep(dftrn, isval=False)
dfval = prep(dfval, isval=True)

In [14]:
dftrn.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,twttyp_TopLevel,twttyp_Retweet,twttyp_Quote,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2,reply,retwt,retwt_cmmnt,like
0,101\t1861\t898\t66921\t11554\t57381\t14018\t16...,73595B00DCB4C616D2ACC3C0BB8385FF\tD2AEE6507AE0...,1046B2F4D16761BFCD00BA8AE45E868A,NaN,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1581233191,6137235B9558D5AE836AF9C66003DA57,...,1,0,0,52141363,223437042,171295679,0,0,0,1
1,101\t56898\t137\t48690\t53504\t11127\t19786\t1...,NaN,260F4BCEAE7443D7497EF4BEEB108E79,NaN,NaN,NaN,Retweet,3A85BCEC571C3F5AB1069E4924189177,1581083472,865439DF5C1A64701A40C55D0B680A23,...,0,1,0,-64538580,106607380,171145960,0,0,0,0
2,101\t865\t75348\t30088\t24667\t98440\t14070\t6...,NaN,B8AFD2889F4B7B0A0C639C5B800EC605,NaN,NaN,NaN,Quote,3A85BCEC571C3F5AB1069E4924189177,1581508481,0A5D33819ADC2E8B3D6631FCF53AC39D,...,0,0,1,-35834649,135736320,171570969,0,0,0,1
3,101\t151\t35976\t865\t53807\t880\t42035\t13432...,NaN,80777BEDE28326A016865438D67798F3,NaN,37BECFDE1D8605DEE7A58AB46859D2CD,3083D4BD7AE85D0817EF4F03D59CFD66,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1580967798,1CB431257DFB41EC9A9995EEE35B2CD1,...,1,0,0,-15102863,29681216,44784079,0,0,0,1
4,101\t115\t38518\t898\t30472\t57381\t57381\t103...,NaN,ED39ECB7ACBCF0C97325AC95A89D1784,Photo\tPhoto\tPhoto\tPhoto,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1581209684,0265B8E3D4FDF64E06246C40D90E66FF,...,1,0,0,-27633626,92871305,120504931,1,0,0,1


In [15]:
dfval.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,has_media_GIF,num_hshtgs,num_links,num_domns,twttyp_TopLevel,twttyp_Retweet,twttyp_Quote,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2
0,101\t108\t880\t14070\t45753\t24667\t860\t45319...,36F8EE6728E9528E2A213A267047735E\tB7AC5EB664BE...,8D7172495C3CDEAC5AFF735C4FACD1CE,NaN,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1581768844,66F1D32362D1C0E0CB5859868448A455,...,False,2,0,0,1,0,0,-29543418,45870660,75414078
1,101\t56898\t137\t26591\t10710\t39554\t26311\t1...,NaN,A98ED0909C914D463AD6B4BD75158414,NaN,NaN,NaN,Retweet,3A85BCEC571C3F5AB1069E4924189177,1581867751,D7E2BD9445BF0A4B977C617FAA33D013,...,False,0,0,0,0,1,0,33775448,109288433,75512985
2,101\t20321\t46619\t898\t11845\t11845\t889\t137...,NaN,2663E07BF74DA4E4E558EA3A11B390E0,NaN,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1582080021,64ABD7488400301BACEDD312E111EC28,...,False,0,0,0,1,0,0,39555418,80063628,40508210
3,101\t59556\t21571\t889\t48797\t50419\t29841\t5...,96F266A7D982CC5606FB4A45DB7C7B6A,D01891C5CF883FF19353B3001FB4A4CD,NaN,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1581928361,926D11F251E3DD307AE304C67746807C,...,False,1,0,0,1,0,0,85549148,88379228,2830080
4,101\t108\t62967\t22650\t22650\t870\t11554\t886...,8FAE72205C7A98D79AF722AC118C1D9A,39CDE58218878E9BEBF5BDDE9D3762CE,Video,NaN,NaN,TopLevel,3A85BCEC571C3F5AB1069E4924189177,1581582245,7877474B83F9B26356552AF9E0526DF2,...,False,1,0,0,1,0,0,-147659737,42842543,190502280


# prepare trn val for a lang

In [16]:
def prep_lang(lang):
    dftrn=dd.read_csv(
        f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
        sep='\x01', header=None, names=cols,)

    dfval=dd.read_csv(
        f'{p_in}/val_{lang}_{valtmstmp}.tsv',
        sep='\x01',header=None,names=cols_val,)

    dftrn = prep(dftrn, isval=False)
    dfval = prep(dfval, isval=True)
    dftrn.to_csv(f'{p_out}/dftrn_{lang}_{trntmstmp}__{PRFX}.tsv',sep='\x01',index=False,single_file=True)
    dfval.to_csv(f'{p_out}/dfval_{lang}_{valtmstmp}__{PRFX}.tsv',sep='\x01',index=False,single_file=True)

In [17]:
%%time
lang='22C448FF81263D4BAF2A176145EE9EAD'
prep_lang(lang)

CPU times: user 17min 53s, sys: 1min 6s, total: 19min
Wall time: 16min 17s


## for multiple langs

In [70]:
# dfval=dd.read_csv(
#     f'{p_in}/val_{valtmstmp}.tsv',
#     sep='\x01',header=None,names=cols_val,)

# lang2cnt=dict(dfval.lang.value_counts().compute())
# ttl=sum(lang2cnt.values())
# lang2perc={k:v/ttl for k,v in lang2cnt.items()}
lang2perc={'D3164C7FBCF2565DDF915B1B3AEFB1DC': 0.4281568150154379,
 '22C448FF81263D4BAF2A176145EE9EAD': 0.15653506511637869,
 '06D61DCBBE938971E1EA0C38BD9B5446': 0.08684409325313776,
 'ECED8A16BE2A5E8871FD55F4842F16B1': 0.07187590645071645,
 'B9175601E87101A984A50F8A62A1C374': 0.0510950651798385,
 '4DC22C3F31C5C43721E6B5815A595ED6': 0.03233621220538451,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 0.02869156970756396,
 '125C57F4FA6D4E110983FB11B52EFD4E': 0.02251587222472389,
 '022EC308651FACB02794A8147AEE1B78': 0.022084940431066645,
 'FA3F382BC409C271E3D6EAF8BE4648DD': 0.020434654769361918,
 '9BF3403E0EB7EA8A256DA9019C0B0716': 0.01711266575901506,
 '975B38F44D65EE42A547283787FF5A21': 0.008900304897960587,
 '2996EB2FE8162C076D070A4C8D6532CD': 0.008491055207128863,
 'FF60A88F53E63000266F8B9149E35AD9': 0.005443463784674508,
 '717293301FE296B0B61950D041485825': 0.005351975887386331,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 0.005259694742433805,
 '3820C29CBCA409A33BADF68852057C4A': 0.005107391190878921,
 '9ECD42BC079C20F156F53CB3B99E600E': 0.0025499607210198202,
 '76B8A9C3013AE6414A3E6012413CDC3B': 0.0025153883436486377,
 'AEF22666801F0A5846D853B9CEB2E327': 0.002179910685601312,
 '190BA7DA361BC06BC1D7E824C378064D': 0.0019262697449259252,
 '1FFD2FE4297F5E70EBC6C3230D95CB9C': 0.0019155609014572224,
 'A0C7021AD8299ADF0C9EBE326C115F6F': 0.0012986125305102882,
 'D413F5FE5236E5650A46FD983AB39212': 0.0012444072734464839,
 '48236EC80FDDDFADE99420ABC9210DDF': 0.0011455818352630845,
 '691890251F2B9FF922BE6D3699ABEFD2': 0.0008832151702798657,
 '920502FAA080485768AA89BC96A55C47': 0.0007246317413822235,
 '0331BF70E606D62D92C96CE9AD71A7CF': 0.0006743927226401609,
 '89616CFF8EC8637092F885C7EFF43D74': 0.0006288470859121594,
 '06BEAB41D66CCFF329D1ED8BA120A6C2': 0.0006205840894085308,
 '60FBA0E834CC59D647C3599AD763FFDF': 0.0005243367061342635,
 'C7A400D9AD489ACF673CF12FBB80AAE5': 0.00047032976098654625,
 'E59EF8BB86A6D815331DDF4C467CE0C7': 0.0004138108649017259,
 '4249CE88433AEA3F8DCEECF008B3CB95': 0.00038637771650967855,
 '544FA32458C903F1125FE6598300A047': 0.00037732147234170147,
 'CB11E9CF42BD0A1BAD5E27BF3422D99D': 0.00033765908912428366,
 '3A85BCEC571C3F5AB1069E4924189177': 0.0002736043402281539,
 'FF7EABB5A382356D54D9C41BA0125E09': 0.00025767328296915775,
 'E7BB61D2A87C1E72DF1C7BC292B86A1C': 0.0002548308121719095,
 '69C4A33B9AD29AF883D60BA61CC08702': 0.00024068456215769712,
 'F4FD40A716F1572C9A28E9CAA58BE3A5': 0.00021701934017130447,
 '259A6F6DFD672CB1F883CBEC01B99F2D': 0.00019963399552766966,
 '6431A618DCF7F4CB7F62A95A39BAB77A': 0.000198113604171002,
 '3121F7240D488F74EEED9312E174B217': 0.00018991671163940232,
 '54208B51D44E7D91DC2F3DD02ADEDEC2': 0.00016354122679981947,
 'DBEEFB80F8A314311E2B4BD593E11DFE': 0.00016235135530329693,
 'B6D90127A09AB1229731898AEF9D4D7C': 0.00013749626182038175,
 'F3E1016563360F9434FA986CA86C249C': 0.0001344554791070464,
 '2216D01F7B48554E4211021A46816FCF': 0.00011581415899486002,
 '1BC639981AE88E09129594B11F894A21': 9.578465547006403e-05,
 'C942E369C88CE7C56E69A84D04319FF0': 5.770876758134292e-05,
 '57ADD4576E2AD6648E9B2DE32F3462A5': 5.446967295192046e-05,
 'AC1F0671A4B0D5B8112F87DE7B490E6D': 4.501680495176922e-05,
 'C2EF5FABE7619D8A333D5F0FF76E1BFA': 4.0587838825824233e-05,
 'AA0254541959271ED3453119B787D0C3': 3.080445096552784e-05,
 'A6B70CDF8C7B934D4A218CA9B6B7FDB4': 1.8971839972331523e-05,
 'BF477808A37E3E4E9C5D9F1839E8519E': 1.2229234825370492e-05,
 '97F81BD92A1ACA3F1F43C154E689350F': 1.1568195105080195e-05,
 '60A3DB168094D41241E45E0DE3539BC0': 1.070884346870281e-05,
 '12D8CEB94F89D11D7EB95EAE9689B009': 7.800268699425504e-06,
 'F73266A79468BB89C4325FDEDB0B533C': 5.552733650438494e-06,
 '4690215948DBF6872B8ED1C2BC87B17E': 5.354421734351405e-06,
 'D18801336202297E6484F634CAC6592E': 3.834030377683722e-06,
 '2E18F6F53E3CF073911AF0A93BBE5373': 2.644158881161188e-07,
 'B2235C8B73239FDC5780DD132419833A': 1.9831191608708906e-07,
 '515E873C86EE1577E75FA2387B7FA59E': 6.61039720290297e-08}

In [72]:
for lang,perc in lang2perc.items():
    try: 
        prep_lang(lang)
        print(dtnow(),lang)
    except Exception as e:
        print(e)

[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_D3164C7FBCF2565DDF915B1B3AEFB1DC_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_22C448FF81263D4BAF2A176145EE9EAD_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_06D61DCBBE938971E1EA0C38BD9B5446_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_ECED8A16BE2A5E8871FD55F4842F16B1_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_B9175601E87101A984A50F8A62A1C374_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_4DC22C3F31C5C43721E6B5815A595ED6_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_167115458A0DBDFF7E9C0C53A83BAC9B_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_125C57F4FA6D4E110983FB11B52EFD4E_1584412344.tsv'
[Errno 2] No such file or directory: '/data/git/recsys20/input/trn_022EC

# prep whole val set

In [15]:
%%time
dfval=dd.read_csv(
    f'{p_in}/val_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)
dfval = prep(dfval, isval=True)
dfval.to_csv(f'{p_out}/dfval_{valtmstmp}__{PRFX}.tsv',sep='\x01',index=False,single_file=True)

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (1,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (1,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


CPU times: user 8min 14s, sys: 35.2 s, total: 8min 50s
Wall time: 7min 33s


['/data/git/recsys20/output/prep0318_2/dfval_1584626395__prep0318_2.tsv']

In [16]:
dfval.columns

Index(['toks', 'hshtgs', 'twtid', 'media', 'links', 'domns', 'twttyp', 'lang',
       'tm', 'u1id', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2id', 'u2_follower_cnt', 'u2_following_cnt',
       'u2_vrfed', 'u2_create_tm', 'u1_fllw_u2', 'len_toks', 'has_media_Photo',
       'has_media_Video', 'has_media_GIF', 'num_hshtgs', 'num_links',
       'num_domns', 'twttyp_TopLevel', 'twttyp_Retweet', 'twttyp_Quote',
       'tmdlta_u2u1', 'tmdlta_twtu1', 'tmdlta_twtu2'],
      dtype='object')